In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *
PATH="/home/kirana/Documents/phd/final"
DATAPATH="/home/kirana/Documents/phd/data/aclImdb"

## Dataset

In [2]:
[df_train,df_valid,df_test,itos, train_tokens, valid_tokens, test_tokens, trn_lm, val_lm, test_lm]=pickle.load(open(f'{PATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [3]:
type(df_train)

pandas.core.frame.DataFrame

In [4]:
df_train=pd.concat([df_train,df_valid],axis=0)

In [5]:
df_train.shape

(75000, 4)

In [6]:
df_valid=df_test.copy()

In [7]:
df_train=df_train.loc[df_train['label']<2]

In [8]:
df_train.shape

(25000, 4)

In [9]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [10]:
type(df_train['tokens'])

pandas.core.series.Series

In [11]:
bs=52 # 52 - Jeremey, 20 - default

In [12]:
bptt= 70 #70 - Jeremey, 35 - default

In [13]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [14]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [15]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [16]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [17]:
padlen=1400
padding_idx=1

In [18]:
df_train['n_tok']=df_train['tokens'].apply(len)

In [19]:
df_train['n_tok'].describe()

count    25000.000000
mean       304.945840
std        224.981807
min         16.000000
25%        166.000000
50%        228.000000
75%        371.000000
max       3354.000000
Name: n_tok, dtype: float64

In [20]:
df_train.sort_values(by='n_tok', inplace=True)

In [21]:
df_valid['n_tok']=df_valid['tokens'].apply(len)
df_valid.sort_values(by='n_tok', inplace=True)

In [22]:
df_test['n_tok']=df_test['tokens'].apply(len)
df_test.sort_values(by='n_tok', inplace=True)

In [23]:
for i in range(0,20):
    x=df_train['tokens'].values[i]
    y=df_train['label'].values[i]    
    out=np.ones(padlen)
    if len(x) < padlen:
        out[:len(x)]=x
    else:
        out=x[:padlen]
    print (out)
    print (y)
    print (len(out))
    if i>10:
        break

[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400


In [24]:
df_train.shape

(25000, 5)

In [25]:
df_train['label'].value_counts(), df_valid['label'].value_counts(),df_test['label'].value_counts()

(1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64)

In [26]:
df_train.shape, df_valid.shape

((25000, 5), (25000, 5))

In [27]:
padlen

1400

In [28]:
class ds_sentiment (Dataset):
    def __init__ (self,df,bs,padlen=64,xvar='tokens',yvar='label',len_var='n_tok',padding_idx=1):
        self.x,self.y,self.padlen,self.padding_idx,self.len_var,self.bs=\
            df[xvar],df[yvar],padlen,padding_idx,df[len_var],bs
        self.len_var=self.len_var.clip(0,padlen)
    
    def pad (self,x):
        out=np.ones(self.padlen)*self.padding_idx
        out=out.astype(int)
        if len(x)>=self.padlen:
            out[:]=x[:self.padlen]
        else:
            out[:len(x)]=x
        return out
        
    def __getitem__(self,idx):
        #import pdb
        #pdb.set_trace()
        return self.pad(self.x.iloc[idx]),self.y.iloc[idx],self.len_var.iloc[idx]
    
    def __len__(self):
        return len(self.x)        


In [29]:
#padlen=max(df_train['n_tok'])
padlen=1400

In [30]:
padlen

1400

In [31]:
dstrain=ds_sentiment(df_train,bs,padlen)
dsvalid=ds_sentiment(df_valid,bs,padlen)
dstest=ds_sentiment(df_test,bs,padlen)

In [32]:
dltrain=DataLoader(dstrain,bs,True)
dlvalid=DataLoader(dsvalid,bs,False)
dltest=DataLoader(dstest,bs,False)

In [33]:
for xb,yb,xlen in dltrain:
    break

In [34]:
xb, yb, xlen

(tensor([[41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         ...,
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1]]),
 tensor([1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1,
         1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
         1, 1, 1, 1]),
 tensor([373, 188, 710, 285, 345, 295, 299,  94,  79, 697, 145, 384, 227, 784,
         414, 324, 610, 231, 532, 229,  98, 550, 231, 257, 207, 223, 181,  82,
         488, 260, 884, 288, 188, 274, 315, 199, 681, 123, 170, 377, 315, 152,
         188, 159, 281, 174, 356, 119, 228, 104, 187, 151]))

## Model Architecture

In [35]:
from torch import nn
import torch
from torch.autograd import Variable

In [36]:
bs,bptt

(52, 70)

In [38]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=400#400
n_layers= 2 # 2
dropout=0.5 # 0.5
wd=1e-7
bidirectional=True
dropout_e=0.1 # 0.5
dropout_o=0.5 #0.5
n_out=1

In [39]:
xb.shape,xlen.shape

(torch.Size([52, 1400]), torch.Size([52]))

In [40]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [41]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [42]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [94]:
class sentiment_classifier (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,n_out=1,padding_idx=1):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx,self.padding_idx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx,padding_idx
        self.n_out=n_out
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        
        self.create_architecture()
        if pretrain_mtx is not None:
            print (f'initializing glove with {pretrain_mtx.shape}')
            self.initialize_glove()
        self.init_hidden()
        self.criterion=nn.BCEWithLogitsLoss()
    
    def set_dropouts(self, dropout, dropout_o, dropout_e):
        self.dropout, self.dropout_o, self.dropout_e = dropout, dropout_o, dropout_e
    
    def create_architecture(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
             # embs are going to be of shape n_batch * n_seq * n_emb
            # h and c are of shape n_layers * n_batch * n_hidden
        # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=self.bidirectional)
        
                # output is of shape n_batch * n_seq * n_hidden
            # output [:,-1,:] is the same as hn[-1]
        self.dropout_op=nn.Dropout(self.dropout_o)
        self.fc=nn.Linear(self.n_hidden*2,self.n_out)

        #self.log_softmax=nn.LogSoftmax()
        self.sigmoid=nn.Sigmoid()
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
         
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True

    def initialize_glove(self):
        self.encoder.weight.data.copy_(torch.Tensor(self.pretrain_mtx))
    
    def init_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    
        
    def forward(self,Xb,Yb,Xb_lengths):
        embs=self.dropout_enc(self.encoder(Xb))
        
        import pdb
        pdb.set_trace()
        
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())

        #import pdb
        #pdb.set_trace()
        # before lstm call
            # pack_padded_sequence so that padded items in the sequence won't be shown to the LSTM
        packed_embs = pack_padded_sequence(embs,Xb_lengths,batch_first=True, enforce_sorted=False)
        lstm_out,(hidden,cell)=self.lstm(packed_embs)

        # after lstm call
        lstm_out,_=pad_packed_sequence(lstm_out,batch_first=True)
        #lstm_out=self.dropout_op(lstm_out)      
        
        # Wrap the hidden state in a new tensor without the gradients
        #self.hidden=(Variable(newhidden[0].data,requires_grad=False).to(self.device),Variable(newhidden[1].data,requires_grad=False).to(self.device))
        
            # lstm_out has the values that are padded
        #out=self.fc (lstm_out[:,-1,:])
        hidden = self.dropout_op(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        
        preds=self.fc(hidden)
        
              
        
        loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))
        
        return preds.view(-1), loss  

In [95]:
def accuracy_binomial(preds,actual, device="cpu", cutoff=0.5):
    preds=torch.sigmoid(preds)
    zeros=torch.zeros(len(preds)).to(device)
    ones = torch.ones(len(preds)).to(device)

    preds=torch.where(preds>cutoff,ones,zeros)
    correct=torch.round(preds).long()==actual
    return correct.float().sum()/len(correct)

In [96]:
def accuracy_binomial(preds, y, device="cpu", cutoff=0.5):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    y=y.float()
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [97]:
device="cpu"

## Load pretrained model weights

In [98]:
pretrained_lm_weights=pickle.load(open(f'{PATH}/inter/pretrained_lm_weights','rb'))

In [99]:
if 1==0:
    model=torch.load (f'{PATH}/inter/model_awd_lstm')
    pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
    import pickle
    pickle.dump(pretrained_lm_weights,open(f'{PATH}/inter/pretrained_lm_weights','wb'))

### Check if model forward works

In [100]:
n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout_o,dropout,n_out

(59972, 400, 400, 2, True, 52, 'cpu', 0.1, 0.5, 0.5, 1)

In [101]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=dropout_e,dropout=dropout,\
                 dropout_o=dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)


initializing glove with (59972, 400)


In [102]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [103]:
print(f'The model has {count_parameters(model_sentiment):,} trainable parameters')

The model has 30,402,401 trainable parameters


In [104]:
model_sentiment

sentiment_classifier(
  (dropout_enc): Dropout(p=0.1)
  (encoder): Embedding(59972, 400, padding_idx=1)
  (lstm): LSTM(400, 400, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout_op): Dropout(p=0.5)
  (fc): Linear(in_features=800, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (criterion): BCEWithLogitsLoss()
)

In [105]:
model_sentiment.forward(xb,yb,xlen)

> <ipython-input-94-5ce529cfac96>(61)forward()
-> if Xb.size(0) < self.bs:
(Pdb) Xb.size()
torch.Size([52, 1400])
(Pdb) n
> <ipython-input-94-5ce529cfac96>(69)forward()
-> packed_embs = pack_padded_sequence(embs,Xb_lengths,batch_first=True, enforce_sorted=False)
(Pdb) embs.size
<built-in method size of Tensor object at 0x7fb99055bea0>
(Pdb) embs.size()
torch.Size([52, 1400, 400])
(Pdb) q


BdbQuit: 

In [55]:
preds,loss=model_sentiment.forward(xb,yb,xlen)

In [56]:
preds.view(-1)

tensor([ 0.0167,  0.0165,  0.0045,  0.0602,  0.0163,  0.0170,  0.0510,  0.0005,
        -0.0033,  0.0505,  0.0200, -0.0091,  0.0041, -0.0191,  0.0200,  0.0176,
         0.0683,  0.0205, -0.0131,  0.0160,  0.0200,  0.0877, -0.0030,  0.0219,
         0.0279, -0.0068,  0.0898,  0.0564, -0.0069,  0.0128,  0.0291,  0.0198,
         0.0516, -0.0233,  0.0676, -0.0441,  0.0772,  0.0030,  0.0120,  0.0826,
        -0.0108,  0.0237,  0.0244,  0.0250, -0.0137,  0.0793, -0.0614, -0.0011,
         0.0488, -0.0047,  0.0209,  0.0993], grad_fn=<ViewBackward>)

In [57]:
yb

tensor([1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1,
        1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
        1, 1, 1, 1])

In [58]:
preds.size()

torch.Size([52])

In [59]:
yb.size()

torch.Size([52])

In [60]:
accuracy_binomial(preds.to(device),yb.to(device))

tensor(0.6154)

In [61]:
from sklearn.metrics import roc_auc_score

In [62]:
roc_auc_score(yb,preds)

0.5424242424242425

In [63]:
pretrained_lm_weights

array([[-9.038397e-02, -1.795423e-02, -3.457930e-02,  8.671192e-02, ..., -3.003510e-02, -1.899482e-02,  1.426260e-02,
        -4.948893e-02],
       [-2.453323e-02, -3.018817e-02,  6.663179e-02, -5.793565e-02, ...,  1.840935e-02,  6.371137e-02,  9.835491e-03,
        -2.131385e-03],
       [ 7.696263e-03, -4.266643e-02, -1.533517e-01,  2.239776e-01, ...,  9.869517e-02,  3.041433e-02,  1.824751e-01,
         1.134978e-01],
       [ 3.305928e-02,  2.266591e-01, -4.264669e-02,  1.490862e-01, ...,  3.407921e-02, -6.422209e-03,  3.180612e-01,
         9.373549e-02],
       ...,
       [ 6.027538e-02, -5.980809e-02,  1.861691e-01, -3.105092e-02, ..., -2.764457e-02,  1.962678e-02, -2.172215e-03,
         6.297247e-02],
       [-1.592789e-02, -2.781571e-04,  1.301994e-01,  2.851282e-02, ...,  5.064877e-02,  1.670864e-01,  2.283701e-02,
        -8.746398e-03],
       [ 4.669700e-02,  3.138980e-02,  1.221957e-02, -3.927753e-02, ..., -1.281436e-01,  1.121320e-01, -3.726090e-03,
        -3.777364e

## Learner

In [64]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
  
        
    
    def fit (self,Xb,Yb,Xlen,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb,Xlen)
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1),self.device)
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_val)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.init_hidden()
        for Xb,Yb,Xlen in iterator:
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            Xlen=Xlen.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,Xlen,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            
            k=k+1
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/len(iterator)
        epoch_acc=epoch_acc/len(iterator)
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

    
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        for epoch in range(n_epochs):
            
            loss,acc=self.run_epoch(dltrain,True)
            print (f'Epoch:{epoch} Loss:{loss}')
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Loss:{loss} Accuracy:{acc} Loss:{lossv} Accuracy:{accv}')
        

In [65]:
device="cuda:1"

In [66]:
n_layers

2

In [67]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout,\
                 dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)
model_sentiment=model_sentiment.to(device)

initializing glove with (59972, 400)


In [68]:
optimizer=torch.optim.Adam(model_sentiment.parameters(),lr=5e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_binomial

In [70]:
len(dltrain),len(dlvalid)

(481, 481)

In [71]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [72]:
learner=Learner(model_sentiment,optimizer,accuracy_binomial,device,bptt,50,0.25)

In [73]:
model_sentiment.freeze_embedding()

In [76]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(False, False)

In [77]:
model_sentiment.dropout_e,model_sentiment.dropout,model_sentiment.dropout_o, learner.model.dropout_e,learner.model.dropout,learner.model.dropout_o

(0.1, 0.5, 0.5, 0.1, 0.5, 0.5)

In [78]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.7154590737819672  0.5230769431591034
Batch:100 0.6804004204273224  0.5759615594148636
Batch:150 0.6777678459882737  0.583846174677213
Batch:200 0.6654617066681385  0.5954807898402215
Batch:250 0.6276922417879105  0.6346923296451569
Batch:300 0.5847871498763562  0.6706410501400629
Batch:350 0.5488624019707953  0.6987362892287118
Batch:400 0.5234935874864459  0.7201442569494247
Batch:450 0.49993864277998606  0.7378632756074269
Epoch:0 Loss:0.4888977264973825
Batch:50 0.26740525782108304  0.8926923453807831
Batch:100 0.266957288607955  0.8940384989976883
Batch:150 0.2628666991988818  0.8970513200759888
Batch:200 0.2646070369333029  0.8942308068275452
Batch:250 0.2674366473555565  0.8927692680358886
Batch:300 0.2714722036818663  0.8905769594510397
Batch:350 0.2731277745962143  0.8897802562373025
Batch:400 0.2745375219359994  0.8894231137633324
Batch:450 0.27785389294226964  0.8882906353473663
Epoch:0 Loss:0.4888977264973825 Accuracy:0.746645638080248 Loss:0.2784661314839385 Accu

In [79]:
model_sentiment.dropout_e,model_sentiment.dropout,model_sentiment.dropout_o, learner.model.dropout_e,learner.model.dropout,learner.model.dropout_o

(0.1, 0.5, 0.5, 0.1, 0.5, 0.5)

In [82]:
for param_group in learner.optimizer.param_groups:
    param_group['weight_decay']=1e-5

In [83]:
model_sentiment.set_dropouts(0.6,0.6,0.2)

In [84]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.23294291988015176  0.9123077309131622
Batch:100 0.24807299382984638  0.9065384995937348
Batch:150 0.24289438540736835  0.9070513188838959
Batch:200 0.24411732655018567  0.9051923456788064
Batch:250 0.24465418934822084  0.9037692687511444
Batch:300 0.24328006505966188  0.9048718335231145
Batch:350 0.24162265498723304  0.9053297088827406
Batch:400 0.2442074350081384  0.903653884679079
Batch:450 0.24405123093061976  0.9037179874049293
Epoch:0 Loss:0.24440386171090628
Batch:50 0.23209561333060263  0.9073077297210693
Batch:100 0.23245390437543392  0.9076923441886902
Batch:150 0.226586623887221  0.9121795245011648
Batch:200 0.22360068261623384  0.9133654212951661
Batch:250 0.22603180292248726  0.9132308070659637
Batch:300 0.2267183451106151  0.9122436277071635
Batch:350 0.22796417140534947  0.9112637746334076
Batch:400 0.22826736476272344  0.9109615765511989
Batch:450 0.230920865436395  0.909743627972073
Epoch:0 Loss:0.24440386171090628 Accuracy:0.9038901715665251 Loss:0.231708012

In [85]:
for param_group in learner.optimizer.param_groups:
    param_group['lr']=5e-4

In [86]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.19978587210178375  0.9265385031700134
Batch:100 0.19149932511150836  0.9280769622325897
Batch:150 0.18728538955251375  0.9302564497788747
Batch:200 0.1894717753306031  0.9294231167435646
Batch:250 0.18720175243914128  0.9311538860797882
Batch:300 0.18408042748769124  0.9321154234806697
Batch:350 0.18364969541983944  0.9319231154237475
Batch:400 0.18404425410553812  0.9314904229342937
Batch:450 0.1839684145152569  0.9308974744213951
Epoch:0 Loss:0.18290053171700102
Batch:50 0.2224332244694233  0.9107692670822144
Batch:100 0.2198754394426942  0.9140384984016419
Batch:150 0.21504020594060422  0.9144872156778971
Batch:200 0.20858425343409182  0.9169231143593788
Batch:250 0.21073045270144938  0.9154615762233734
Batch:300 0.21184344183653592  0.9153205502033234
Batch:350 0.21524977301912648  0.9148352023533413
Batch:400 0.2160103526059538  0.9150961916148662
Batch:450 0.21859148723383745  0.9141880722840627
Epoch:0 Loss:0.18290053171700102 Accuracy:0.9312730277898158 Loss:0.219206

In [87]:
torch.save(model_sentiment.state_dict(),f'{PATH}/inter/sentiment_model_nounfreeze_state_dict_0.917')
torch.save(optimizer.state_dict(),f'{PATH}/inter/sentiment_learner_nounfreeze_state_dict_0.917')
torch.save (model_sentiment,f'{PATH}/inter/sentiment_model_nounfreeze_0.917')
torch.save (optimizer,f'{PATH}/inter/sentiment_optimizer_nounfreeze_0.917')
torch.save (learner,f'{PATH}/inter/sentiment_learner_nounfreeze_0.917')

/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type sentiment_classifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


model_sentiment=torch.load(f'{PATH}/inter/sentiment_model_nounfreeze')
optimizer=torch.load(f'{PATH}/inter/sentiment_optimizer_nounfreeze')
learner=torch.load(f'{PATH}/inter/sentiment_learner_nounfreeze')

In [88]:
model_sentiment.unfreeze_embedding()

In [90]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(True, True)

In [91]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.1757665029168129  0.9338461911678314
Batch:100 0.17884126387536525  0.9323077297210693
Batch:150 0.17157089300453662  0.9365384995937347
Batch:200 0.1665740057453513  0.9388461914658547
Batch:250 0.16849402721226214  0.9384615759849548
Batch:300 0.16642002327988545  0.9392949092388153
Batch:350 0.16435486163944005  0.9398901465960912
Batch:400 0.1623965109186247  0.9409615746140481
Batch:450 0.16075962524033255  0.9414102921220991
Epoch:0 Loss:0.16170276429217595
Batch:50 0.22108415484428406  0.9123077321052552
Batch:100 0.21340429924428464  0.9176923459768296
Batch:150 0.20557850728432336  0.9205128566424052
Batch:200 0.1989787089265883  0.9236538824439049
Batch:250 0.2013720786422491  0.921923113822937
Batch:300 0.2017382326349616  0.9211538823445639
Batch:350 0.20419149936309883  0.9204945426327842
Batch:400 0.204031760757789  0.9209134986996651
Batch:450 0.20522147292064297  0.9203419176737467
Epoch:0 Loss:0.16170276429217595 Accuracy:0.9410163481369336 Loss:0.2046149781

In [92]:
for param_group in learner.optimizer.param_groups:
    param_group['lr']=5e-5

In [93]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.11615589462220668  0.966538497209549
Batch:100 0.10260744299739599  0.9701923406124116
Batch:150 0.10320262432098389  0.9685897755622864
Batch:200 0.10043475738726557  0.9695192635059356
Batch:250 0.09697593061625957  0.9700769543647766
Batch:300 0.09517815880477429  0.97019233862559
Batch:350 0.09467971860031997  0.9702747559547424
Batch:400 0.09466664283769205  0.9705769535899162
Batch:450 0.09382010911901792  0.9706837913725112
Epoch:0 Loss:0.09396658818045797
Batch:50 0.24926649011671542  0.9138461947441101
Batch:100 0.2419789445772767  0.9176923471689225
Batch:150 0.23338562476138275  0.9180769606431325
Batch:200 0.2237020975537598  0.9211538836359978
Batch:250 0.22683872967958452  0.9197692685127258
Batch:300 0.22746217250823975  0.9194872166713078
Batch:350 0.23043268159031868  0.9186264107908522
Batch:400 0.23017218199558556  0.9188942676782608
Batch:450 0.2322108591016796  0.9183333708180321
Epoch:0 Loss:0.09396658818045797 Accuracy:0.9705901470600691 Loss:0.2312833

In [89]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.09524211082607507  0.9680769574642182
Batch:100 0.093828199878335  0.9686538773775101
Batch:150 0.0919878846531113  0.9687179811795552
Batch:200 0.09395078460685909  0.9689423388242722
Batch:250 0.09284830520302058  0.9690769546031952
Batch:300 0.09200737948218982  0.9695513131221135
Batch:350 0.09359771828566278  0.9695055254868099
Batch:400 0.09153297192882746  0.9706731064617634
Batch:450 0.09010110232565138  0.971068405840132
Epoch:0 Loss:0.09070706779714248
Batch:50 0.23427742831408976  0.9230769610404969
Batch:100 0.22739623751491309  0.9246154230833054
Batch:150 0.22029945831745862  0.9264102943738302
Batch:200 0.21663093800656497  0.9272115764021873
Batch:250 0.216403607301414  0.927153883934021
Batch:300 0.2173488519154489  0.9263461919625601
Batch:350 0.21949118124055012  0.9253846536363874
Batch:400 0.2178897394472733  0.92600965321064
Batch:450 0.2196268634787864  0.9252137135134803
Epoch:0 Loss:0.09070706779714248 Accuracy:0.9704582101342089 Loss:0.2193625805214

In [179]:
len(dltrain)

433